In [15]:
import pandas as pd
from models.utils.sp_scraper import scrape_sp500_symbols
import yfinance as yf
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, BatchNormalization, LeakyReLU, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential
import talib as ta
import time


In [16]:
# Replace '.' with '-' in ticker symbols, also add SPY as a benchmark
sp_tickers = [ticker.replace(".", "-") for ticker in sorted(scrape_sp500_symbols())] + ["^GSPC"]
data = pd.DataFrame()

# Initialize scaler dictionaries to store scalers for each ticker
scalers = {}

for ticker in tqdm(sp_tickers, desc = "Downloading data", unit="ticker"):
    # Initialize scalers
    scaler_close = StandardScaler()
    scaler_future_price = StandardScaler()
    scaler_sma_10 = StandardScaler()
    scaler_sma_30 = StandardScaler()


    def get_ticker_data(ticker):
        while True:
            try:
                # Get max data for the ticker
                ticker_data = yf.Ticker(ticker).history(period="1y")
                return ticker_data
            except Exception as e:
                time.sleep(10)
                # Continue the loop to try again

    # Usage - this will keep trying until successful
    ticker_data = get_ticker_data(ticker)

    # Make date a column instead of index
    ticker_data.reset_index(inplace=True)

    # Make columns lowercase
    ticker_data.columns = ticker_data.columns.str.lower()

    # Add a ticker column keep uppercase
    ticker_data['ticker'] = ticker

    # Add a price return in 30 days column
    ticker_data['log_return_30d'] = np.log(ticker_data['close'].shift(-30) / ticker_data['close'])

    # Calculate RSI (Relative Strength Index)
    ticker_data['rsi'] = ta.RSI(ticker_data['close'], timeperiod=14)

    # Calculate MACD (Moving Average Convergence Divergence)
    macd, macdsignal, macdhist = ta.MACD(ticker_data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    ticker_data['macd'] = macd  # MACD line

    # Calculate SMA (Simple Moving Average) for 10 and 30 periods
    ticker_data['sma_10'] = ta.SMA(ticker_data['close'], timeperiod=10)
    ticker_data['sma_30'] = ta.SMA(ticker_data['close'], timeperiod=30)

    # Scale close column
    stock_close = ticker_data.filter(["close"])
    stock_log_return_30d = ticker_data.filter(["log_return_30d"])
    stock_sma_10 = ticker_data.filter(["sma_10"])   
    stock_sma_30 = ticker_data.filter(["sma_30"])
    # Convert to numpy array
    stock_close = stock_close.values
    stock_log_return_30d = stock_log_return_30d.values
    stock_sma_10 = stock_sma_10.values
    stock_sma_30 = stock_sma_30.values
    # Scale the data
    scaled_close = scaler_close.fit_transform(stock_close)
    scaled_log_return_30d = scaler_future_price.fit_transform(stock_log_return_30d)
    scaled_sma_10 = scaler_sma_10.fit_transform(stock_sma_10)
    scaled_sma_30 = scaler_sma_30.fit_transform(stock_sma_30)
    # Insert scaled data into the original dataframe
    ticker_data['scaled_close'] = scaled_close
    ticker_data['scaled_log_return_30d'] = scaled_log_return_30d
    ticker_data['scaled_sma_10'] = scaled_sma_10
    ticker_data['scaled_sma_30'] = scaled_sma_30

    # Store the scalers for the ticker
    scalers[ticker] = {
        'scaler_close': scaler_close,
        'scaler_future_price': scaler_future_price,
    }

    # Concat the ticker data with the main data
    data = pd.concat([data, ticker_data], ignore_index=True)

data.dropna(inplace=True)

In [17]:
# Prepare the data for the model
# Label encode the ticker column
label_encoder = LabelEncoder()
data["encoded_ticker"] = label_encoder.fit_transform(data["ticker"])

# Initialize scalers
scaler_ticker = StandardScaler()
scaler_technical = StandardScaler()


# Scale the ticker column
stock_ticker = data.filter(["encoded_ticker"])
stock_ticker = stock_ticker.values
scaled_ticker = scaler_ticker.fit_transform(stock_ticker)
data['scaled_ticker'] = scaled_ticker

#scale technical columns
stock_technical = data.filter(["return", "rsi", "macd"])
stock_technical = stock_technical.values
scaled_technicals = scaler_technical.fit_transform(stock_technical)
# Insert scaled data into the original dataframe
data['scaled_rsi'] = scaled_technicals[:, 0]
data['scaled_macd'] = scaled_technicals[:, 1]

# Group the data by ticker
grouped_dfs = data.groupby('ticker')

In [18]:
# List the features you want to include (excluding 'price in 30 days' and 'date')
feature_cols = ['scaled_close', 'scaled_rsi', 'scaled_macd', 'scaled_sma_10', 'scaled_sma_30', 'scaled_ticker']  # add your actual column names here

x_train, y_train = [], []

for ticker, df in tqdm(grouped_dfs, desc="Creating sliding windows", unit="ticker"):
    df.sort_values(by='date', inplace=True)

    for i in range(60, len(df) - 30):
        # Extract a sliding window of all desired features
        window = df.iloc[i - 60:i][feature_cols].values  # shape (60, num_features)

        # Optional: add ticker as a numeric value if it's useful
        # ticker_id = your_ticker_encoding[ticker]  # if you're using one-hot or label encoding
        # ticker_column = np.full((60, 1), ticker_id)
        # window = np.hstack((window, ticker_column))

        x_train.append(window)

        # Predict the "price in 30 days" from the current i-th index (i.e. day 60 of the window)
        y_train.append(df.iloc[i]['scaled_log_return_30d'])
        
# Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

Creating sliding windows: 100%|██████████| 504/504 [00:10<00:00, 46.26ticker/s]


In [ ]:
model = Sequential()

# Convolutional layers for local pattern extraction
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Stacked Bidirectional LSTM for capturing sequence relationships
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))

# Dense layers for final nonlinear transformation
model.add(Dense(128, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.4))  # Slightly increased dropout to reduce overfitting

model.add(Dense(64, kernel_regularizer=regularizers.l2(0.001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.3))

model.add(Dense(1))  # Output log return prediction

model.summary()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0),
    loss=keras.losses.Huber(delta=1.0),  # Huber = better for stability on noisy targets
    metrics=[keras.metrics.RootMeanSquaredError()]
)


lr_schedule = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                factor=0.5, 
                                                patience=3, 
                                                verbose=1)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                           patience=10, 
                                           restore_best_weights=True)

training = model.fit(
    x_train, y_train,
    epochs=200,                # Max number of epochs
    batch_size=32,
    validation_split=0.1,      # Use part of training data for validation
    callbacks=[early_stopping]
)
model.save("model3.keras")

/Users/aryanhazra/Downloads/VSCode Repos/trading_model/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/aryanhazra/Downloads/VSCode Repos/trading_model/.venv/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 60, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 60, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 30, 128)        │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 203,777 (796.00 KB)

 Trainable params: 203,521 (795.00 KB)

 Non-trainable params: 256 (1.00 KB)

Epoch 1/200
1359/1388 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - loss: 0.5212 - root_mean_squared_error: 0.8903